<a href="https://colab.research.google.com/github/deb-kit2/DeepDream/blob/master/DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Dependencies

In [1]:
from keras.applications import inception_v3
from keras import backend as k

k.set_learning_phase(0)

import numpy as np
import scipy
from keras.preprocessing.image import load_img, save_img, img_to_array

Using TensorFlow backend.


###hello


In [0]:
some_layers= {
    "mixed2" : 0.2,
    "mixed3" : 0.5,
    "mixed4" : 2.,
    "mixed5" : 1.5, 
}

In [3]:
model = inception_v3.InceptionV3(include_top = False, weights = "imagenet")

loss = k.variable(0.)

for layer, coeff in some_layers.items():
  activation = model.get_layer(layer).output
  scaling = k.prod(k.cast(k.shape(activation), 'float32'))
  if k.image_data_format() == 'channels_first':
    loss = loss + coeff * k.sum(k.square(activation[:, :, 2: -2, 2: -2]))/scaling
  else:
    loss = loss + coeff * k.sum(k.square(activation[:, 2: -2, 2: -2, :]))/scaling

In [0]:
my_image = model.input

gradient = k.gradients(loss, my_image)[0]
gradient /= k.maximum(k.mean(k.abs(gradient)), k.epsilon())

fetch = k.function([my_image], [loss, gradient])

def ascent(x, step, iterations, maxLoss = None):
  for i in range(iterations):
    l, g = fetch([x])
    if maxLoss is not None and l > maxLoss:
      break
    if(i%5 == 0):
      print("loss for " + str(i) + "th iteration : " + str(l))
    x += step*g
  return x

###Helpers

In [0]:
def resize(img, shaper):
  if k.image_data_format() == "channels_first":
    zoom = (1, 1, float(shaper[0]/img.shape[2]) , float(shaper[1]/img.shape[3]))
  else:
    zoom = (1, float(shaper[0]/img.shape[1]) , float(shaper[1]/img.shape[2]), 1)
  
  return scipy.ndimage.zoom(img, zoom, order = 1)

In [0]:
def preprocess(path):
  img = load_img(path)
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = inception_v3.preprocess_input(img)
  return img

In [0]:
def deprocess(img):
  if k.image_data_format() == "channels_first":
    img = img.reshape((3, img.shape[2], img.shape[3]))
    img = img.transpose((1, 2, 0))
  else:
    img = img.reshape((img.shape[1], img.shape[2], 3))
  
  img /= 2
  img += 0.5
  img *= 255
  img = np.clip(img, 0., 255.).astype('uint8')
  return img

In [0]:
def save(fname, img):
  temp = np.copy(img)
  image = deprocess(temp)
  imageio.imwrite(fname, image)
  #scipy.misc.imsave(fname, image)

###deepDream

In [0]:
step= 0.01
num_octaves = 3
octave_scale = 1.4
iterations = 20
maxLoss = 10

import os
work_dir = "/content/"
os.chdir("/content/")
path = "Me.jpg"

In [10]:
image = preprocess(path)
successive_shapes = []

if k.image_data_format() == "channels_first":
  original_shape = image.shape[2:]
else:
  original_shape = image.shape[1:3]

successive_shapes.append(original_shape)

for i in range(1, num_octaves):
  shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes.reverse()

original_image = np.copy(image)
shrunk_original = resize(original_image, successive_shapes[0])

for shape in successive_shapes:
  image = resize(image, shape)
  image = ascent(image, step, iterations, maxLoss)
  upscaled_shrunk = resize(shrunk_original, shape)
  upscaled_original = resize(original_image, shape)

  loss_in_detail = upscaled_original - upscaled_shrunk
  image = image + loss_in_detail

  shrunk_original = resize(original_image, shape)

import imageio
save("image_test.png", image)

loss for 0th iteration : 0.8331488
loss for 5th iteration : 2.4415298
loss for 10th iteration : 4.0374527
loss for 15th iteration : 5.382182
loss for 0th iteration : 1.5455457
loss for 5th iteration : 4.492249
loss for 10th iteration : 6.3268776
loss for 15th iteration : 7.808008
loss for 0th iteration : 1.7099823
loss for 5th iteration : 4.952117
loss for 10th iteration : 6.9715266
loss for 15th iteration : 8.545008
